# Modeling Tourism Activity

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn import preprocessing
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [2]:
x = np.load("../data/processed/pca.pickle",allow_pickle=True)
y_df = pd.read_csv("../data/processed/Full_clean.csv")[['ST.INT.ARVL',
        'ST.INT.XPND.MP.ZS', 'ST.INT.XPND.CD',
       'ST.INT.DPRT', 'ST.INT.RCPT.XP.ZS', 'ST.INT.RCPT.CD',
       'Tourist Defecit', 'Tourism Net', 'Tourist Avg Net',
       'Population Estimate', 'ST.INT.ARVL.PER.CAPITA',
       'ST.INT.DPRT.PER.CAPITA']]

In [3]:
# Impute the missing values
y_df.fillna(y_df.median(),inplace=True)

In [4]:
# scale the data
scaler = preprocessing.MinMaxScaler()
y_df[:] = scaler.fit_transform(y_df)

In [5]:
Models = {}

## Linear Regression

In [6]:
linear_regression_models = {}

### OLS

In [7]:
ols = {}
lmX = sm.add_constant(x)

In [8]:
for y in y_df.columns:
    model = sm.OLS(y_df[y].values, lmX)
    results = model.fit()
    ols[y] = results

In [9]:
linear_regression_models["OLS"] = ols

### RecursiveLS

In [10]:
# rls = {}
# for y in y_df.columns:
#     model = sm.RecursiveLS(y_df[y].values, lmX)
#     results = model.fit()
#     rls[y] = results
# linear_regression_models["RLS"] = rls
# for key in rls.keys():
#     print(key, rls[key].rsquared)

## Logit